In [30]:
import pandas as pd
from __future__ import division
import csv
from datetime import datetime
from collections import Counter
import numpy as np
import gmaps

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt')

In [4]:
data.columns = data.columns.str.replace(' ','')

In [5]:
data["datetime"] = data["DATE"] + ' ' + data["TIME"]

In [6]:
data.datetime = pd.to_datetime(data.datetime)

In [7]:
data.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277,2016-03-26 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282,2016-03-26 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282,2016-03-26 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282,2016-03-26 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282,2016-03-26 16:00:00


In [8]:
data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS
data['dTOTAL'] = data.dENTRIES+ data.dEXITS

In [9]:
list(data.columns.values)

['C/A',
 'UNIT',
 'SCP',
 'STATION',
 'LINENAME',
 'DIVISION',
 'DATE',
 'TIME',
 'DESC',
 'ENTRIES',
 'EXITS',
 'datetime',
 'dENTRIES',
 'dEXITS',
 'dTOTAL']

In [10]:
data.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime,dENTRIES,dEXITS,dTOTAL
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277,2016-03-26 00:00:00,0.0,5.0,5.0
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282,2016-03-26 04:00:00,0.0,0.0,0.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282,2016-03-26 08:00:00,0.0,0.0,0.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282,2016-03-26 12:00:00,0.0,0.0,0.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282,2016-03-26 16:00:00,0.0,0.0,0.0


In [11]:
mask = (data.TIME == '00:00:00') | (data.TIME=='01:00:00')

In [12]:
datatest = data.ix[mask, :]

In [13]:
mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)

In [14]:
datatest2 = datatest.ix [mask2, :]

In [15]:
datatest2.shape

(28269, 15)

In [16]:
final = datatest2.groupby([datatest2.DATE,datatest2.STATION]).sum()

In [17]:
mask3 = (datatest2.dEXITS >= 0) & (datatest2.dEXITS <=5000)

In [18]:
final.shape

(2440, 5)

In [19]:
df = final.reset_index()
#df = final.add_suffix('_Count').reset_index()
#Changing to data frame

In [20]:
df.head()

,DATE,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,dTOTAL
0,03/26/2016,1 AV,1409009188,1462205971,1848.0,1050.0,2898.0
1,03/26/2016,103 ST,152097386,594017455,288.0,380.0,668.0
2,03/26/2016,103 ST-CORONA,62520085,59867349,238.0,1266.0,1504.0
3,03/26/2016,104 ST,16816556,15095950,36.0,114.0,150.0
4,03/26/2016,110 ST,26187023,26772943,280.0,682.0,962.0


In [21]:
df.sort(['dTOTAL'], ascending=False, inplace=True)
#sorting by descending order

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [22]:
df.head(10)

,DATE,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,dTOTAL
326,03/26/2016,TIMES SQ-42 ST,3825096876,3080925551,7205.0,1990.0,9195.0
674,03/27/2016,TIMES SQ-42 ST,3825167957,3080996667,6887.0,1849.0,8736.0
56,03/26/2016,34 ST-HERALD SQ,1597992066,1303043366,3667.0,1458.0,5125.0
14,03/26/2016,14 ST-UNION SQ,1415380109,500640412,3640.0,1143.0,4783.0
2418,04/01/2016,TIMES SQ-42 ST,3825616648,3081404086,3681.0,1086.0,4767.0
58,03/26/2016,34 ST-PENN STA,2508669961,852711655,2851.0,1774.0,4625.0
405,03/27/2016,34 ST-HERALD SQ,1598065541,1303108564,3092.0,1357.0,4449.0
358,03/27/2016,125 ST,643870177,523483047,1935.0,2409.0,4344.0
9,03/26/2016,125 ST,643817979,523438192,1878.0,2429.0,4307.0
407,03/27/2016,34 ST-PENN STA,2508733787,852780381,2549.0,1725.0,4274.0


In [23]:
df['Weekend'] = 0
df['Weekend'][df['DATE'] == ('03/26/2016')] = 1
df['Weekend'][df['DATE'] == ('03/27/2016')] = 1
#Creating new Weekend column.  For Sat Mid - 4 AM and Sun Mid - 4 AM

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [24]:
df.head(100)

,DATE,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,dTOTAL,Weekend
326,03/26/2016,TIMES SQ-42 ST,3825096876,3080925551,7205.0,1990.0,9195.0,1
674,03/27/2016,TIMES SQ-42 ST,3825167957,3080996667,6887.0,1849.0,8736.0,1
56,03/26/2016,34 ST-HERALD SQ,1597992066,1303043366,3667.0,1458.0,5125.0,1
14,03/26/2016,14 ST-UNION SQ,1415380109,500640412,3640.0,1143.0,4783.0,1
2418,04/01/2016,TIMES SQ-42 ST,3825616648,3081404086,3681.0,1086.0,4767.0,0
58,03/26/2016,34 ST-PENN STA,2508669961,852711655,2851.0,1774.0,4625.0,1
405,03/27/2016,34 ST-HERALD SQ,1598065541,1303108564,3092.0,1357.0,4449.0,1
358,03/27/2016,125 ST,643870177,523483047,1935.0,2409.0,4344.0,1
9,03/26/2016,125 ST,643817979,523438192,1878.0,2429.0,4307.0,1
407,03/27/2016,34 ST-PENN STA,2508733787,852780381,2549.0,1725.0,4274.0,1


In [25]:
trim_df = df.head(10)

In [27]:
# import numpy as np

# station = trim_df['STATION']
# values = trim_df['dTOTAL_Count']

# plt.figure()

# plt.barh(np.arange(len(station)), values)

# plt.yticks(np.arange(len(station)),
#            ['Station {}'.format(x) for x in station])

# plt.show()

In [54]:
df

,DATE,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,dTOTAL,Weekend,Station Location
326,03/26/2016,TIMES SQ-42 ST,3825096876,3080925551,7205.0,1990.0,9195.0,1,[]
674,03/27/2016,TIMES SQ-42 ST,3825167957,3080996667,6887.0,1849.0,8736.0,1,[]
56,03/26/2016,34 ST-HERALD SQ,1597992066,1303043366,3667.0,1458.0,5125.0,1,[]
14,03/26/2016,14 ST-UNION SQ,1415380109,500640412,3640.0,1143.0,4783.0,1,[]
2418,04/01/2016,TIMES SQ-42 ST,3825616648,3081404086,3681.0,1086.0,4767.0,0,[]
58,03/26/2016,34 ST-PENN STA,2508669961,852711655,2851.0,1774.0,4625.0,1,[]
405,03/27/2016,34 ST-HERALD SQ,1598065541,1303108564,3092.0,1357.0,4449.0,1,[]
358,03/27/2016,125 ST,643870177,523483047,1935.0,2409.0,4344.0,1,[]
9,03/26/2016,125 ST,643817979,523438192,1878.0,2429.0,4307.0,1,[]
407,03/27/2016,34 ST-PENN STA,2508733787,852780381,2549.0,1725.0,4274.0,1,[]


In [50]:
df_locations = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv')
df_locations = df_locations[['Station Name', 'Station Latitude', 'Station Longitude']]
df_locations_upper = df_locations
df_locations_upper['Station Name'] = df_locations['Station Name'].str.upper()
df.merge(df_locations_upper, left_on='STATION', right_on='Station Name', how='outer')

,Station Name,Station Latitude,Station Longitude
0,25TH ST,40.660397,-73.998091
1,25TH ST,40.660397,-73.998091
2,36TH ST,40.655144,-74.003549
3,36TH ST,40.655144,-74.003549
4,36TH ST,40.655144,-74.003549
5,45TH ST,40.648939,-74.010006
6,45TH ST,40.648939,-74.010006
7,45TH ST,40.648939,-74.010006
8,45TH ST,40.648939,-74.010006
9,53RD ST,40.645069,-74.014034


In [38]:
df['Station Location'] = [[] for x in range(len(df))]

In [40]:
df['Station Location']

326     []
674     []
56      []
14      []
2418    []
Name: Station Location, dtype: object

In [53]:
df.merge(df_locations_upper, left_on='STATION', right_on='Station Name', how='outer')

,DATE,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,dTOTAL,Weekend,Station Location,Station Name,Station Latitude,Station Longitude
0,03/26/2016,TIMES SQ-42 ST,3.825097e+09,3.080926e+09,7205.0,1990.0,9195.0,1.0,[],NaN,NaN,NaN
1,03/27/2016,TIMES SQ-42 ST,3.825168e+09,3.080997e+09,6887.0,1849.0,8736.0,1.0,[],NaN,NaN,NaN
2,04/01/2016,TIMES SQ-42 ST,3.825617e+09,3.081404e+09,3681.0,1086.0,4767.0,0.0,[],NaN,NaN,NaN
3,03/31/2016,TIMES SQ-42 ST,3.707741e+09,3.080593e+09,3252.0,794.0,4046.0,0.0,[],NaN,NaN,NaN
4,03/28/2016,TIMES SQ-42 ST,3.825223e+09,3.081047e+09,3000.0,943.0,3943.0,0.0,[],NaN,NaN,NaN
5,03/29/2016,TIMES SQ-42 ST,3.825313e+09,3.081128e+09,2736.0,904.0,3640.0,0.0,[],NaN,NaN,NaN
6,03/30/2016,TIMES SQ-42 ST,3.825412e+09,3.081219e+09,2683.0,796.0,3479.0,0.0,[],NaN,NaN,NaN
7,03/26/2016,34 ST-HERALD SQ,1.597992e+09,1.303043e+09,3667.0,1458.0,5125.0,1.0,[],NaN,NaN,NaN
8,03/27/2016,34 ST-HERALD SQ,1.598066e+09,1.303109e+09,3092.0,1357.0,4449.0,1.0,[],NaN,NaN,NaN
9,04/01/2016,34 ST-HERALD SQ,1.598618e+09,1.303596e+09,2357.0,931.0,3288.0,0.0,[],NaN,NaN,NaN


In [42]:
for row in df:
    for row_loc in df_locations:
        print row_loc
#         if row_loc['Station Name'] == row['STATION']:
#             row['Station Location'].append([row_loc['Station Latitude'],row_loc['Station Longitude']])

Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude
Station Name
Station Latitude
Station Longitude


In [ ]:
def Jitter(values, jitter=0.000001):
    n = len(values)
    return np.random.uniform(-jitter, +jitter) + values